# Oscilloscope level trigger

### StemLab 10 bit board (DAQ) configuration:
- Both input channels are configured with the jumper in LV position (high sensitivity, $\pm$ 1 V)
- Visit Red Pitaya IP (dashboard) > Development > Python Programming

----------------------------------

Simple notebook for acquisition of a single event, this can be used for parameter fine tuning. The parameters can then be import into the Continuous_acquisition notebook

----------------------------------



In [ ]:
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

### Initial configuration

In [ ]:
# Import the FPGA firmware Overlay
from redpitaya.overlay.mercury import mercury as overlay
fpga = overlay()

# Import LEDs
LED = FPGA.led
aq_started = LED(1, 0)
trig_armed = LED(2, 0)
event      = LED(3, 0)

# Import GPIOs for external event counter / slow trigger
GPIO = FPGA.gpio
gpio_o = GPIO('p', 0, "out")

# Define the FPGA oscilloscope objects
osc = [fpga.osc(ch, 1.0) for ch in range(2)]

# Set data rate decimation
for ch in osc:
    # data rate decimation = 1, keep 125 MHz sampling frequency
    ch.decimation = 1

    # trigger timing [sample periods]
    N = ch.buffer_size
    ch.trigger_pre  = 0
    ch.trigger_post = N
    
    # osc[0] is controlling both channels
    ch.sync_src = fpga.sync_src["osc0"]

# Setup trigger on scintillator signals
osc[0].level = [0.23, 0.2]
osc[0].edge  = 'pos'
osc[1].level = [0.23, 0.2]
osc[1].edge  = 'pos'

for ch in osc:
    ch.trig_src = fpga.trig_src["osc1"]

### Single event acquisition

In [ ]:
aq_started.write(1)

# reset and start
osc[0].reset()
osc[0].start()
# wait for trigger
trig_armed.write(1)
# Reset the slow GPIO output for external event counting
gpio_o.write(False)
while (osc[0].status_run()): pass
# "fast" (code execution time) pulse on the slow GPIO output for slow trigger and external event counting with scaler
gpio_o.write(True)
trig_armed.write(0)
timestamp = time.time()
print('Recorded event', " timestamp = " ,str(timestamp))
data0 = osc[0].data(N)
data1 = osc[1].data(N)
time.sleep(0.5)
gpio_o.write(False)

#Single event visualization
buf_len0 = len(data0)
buf_len1 = len(data1)
buf_time0 = np.linspace(0, float(buf_len0) * (1./0.125), buf_len0)
buf_time1 = np.linspace(1, float(buf_len1) * (1./0.125), buf_len1)

fig, ax = plt.subplots()
ax.plot(buf_time0, data0, color = 'red',  label = 'Scint_0')
ax.plot(buf_time1, data1, color = 'blue', label = 'Scint_1')
ax.grid()
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Signal Amplitude mV")
ax.title("Event signal shape")